## Part1

year                
month    
day_of_month
day_of_week

fl_date

In [131]:
import numpy as np
import pandas as pd

In [132]:
# path = r"D:\py\5243\project1\flight_data_2024.csv"
# df = pd.read_csv(path)

df = pd.read_csv("data/flight_data_2024.csv")

cols1 = ["year", "month", "day_of_month", "day_of_week", "fl_date"]

/var/folders/q4/t7g5qths559gsjx7h006w5w40000gn/T/ipykernel_7816/548968178.py:4: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/flight_data_2024.csv")


In [133]:
df1=df.copy()

In [134]:
df.info() # get basic info for dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7079081 entries, 0 to 7079080
Data columns (total 35 columns):
 #   Column               Dtype  
---  ------               -----  
 0   year                 int64  
 1   month                int64  
 2   day_of_month         int64  
 3   day_of_week          int64  
 4   fl_date              object 
 5   op_unique_carrier    object 
 6   op_carrier_fl_num    float64
 7   origin               object 
 8   origin_city_name     object 
 9   origin_state_nm      object 
 10  dest                 object 
 11  dest_city_name       object 
 12  dest_state_nm        object 
 13  crs_dep_time         int64  
 14  dep_time             float64
 15  dep_delay            float64
 16  taxi_out             float64
 17  wheels_off           float64
 18  wheels_on            float64
 19  taxi_in              float64
 20  crs_arr_time         int64  
 21  arr_time             float64
 22  arr_delay            float64
 23  cancelled            int64  
 24

In [135]:
# check missing values for each column
print("Missing values per column:")
print(df.isna().sum())

print("\nNumber of duplicated rows:")
print(df.duplicated().sum())


Missing values per column:
year                         0
month                        0
day_of_month                 0
day_of_week                  0
fl_date                      0
op_unique_carrier            0
op_carrier_fl_num            1
origin                       0
origin_city_name             0
origin_state_nm              0
dest                         0
dest_city_name               0
dest_state_nm                0
crs_dep_time                 0
dep_time                 92659
dep_delay                92970
taxi_out                 95734
wheels_off               95734
wheels_on                97856
taxi_in                  97856
crs_arr_time                 0
arr_time                 97854
arr_delay               113814
cancelled                    0
cancellation_code      6982766
diverted                     0
crs_elapsed_time             1
actual_elapsed_time     113814
air_time                113814
distance                     0
carrier_delay                0
weather_dela

In [136]:
print("Missing values:")
print(df[cols1].isna().sum())

Missing values:
year            0
month           0
day_of_month    0
day_of_week     0
fl_date         0
dtype: int64


In [137]:
df.shape

(7079081, 35)

In [138]:
abnormal_month = df[(df["month"] < 1) | (df["month"] > 12)]
abnormal_day = df[(df["day_of_month"] < 1) | (df["day_of_month"] > 31)]
abnormal_week = df[(df["day_of_week"] < 1) | (df["day_of_week"] > 7)]

print("\nAbnormal month:", len(abnormal_month))
print("Abnormal day_of_month:", len(abnormal_day))
print("Abnormal day_of_week:", len(abnormal_week))


Abnormal month: 0
Abnormal day_of_month: 0
Abnormal day_of_week: 0


In [139]:
df["fl_date"] = pd.to_datetime(df["fl_date"], errors="coerce")

print("\nfl_date parse failed:")
print(df["fl_date"].isna().sum())


fl_date parse failed:
0


In [140]:
rebuild = pd.to_datetime(
    {
        "year": df["year"],
        "month": df["month"],
        "day": df["day_of_month"]
    },
    errors="coerce"
)

mask = df["fl_date"].notna() & rebuild.notna()

mismatch = df.loc[mask & (df["fl_date"] != rebuild)]

print("Date mismatch rows:", len(mismatch))


Date mismatch rows: 0


## Part 2

op_unique_carrier

op_carrier_fl_num

origin

origin_city_name

origin_state_nm

dest

dest_city_name

dest_state_nm

## Part 2-1

In [141]:
# varible: op_unique_carrier

s = df["op_unique_carrier"].astype("string").str.strip()
mask_bad = s.isna() | (s.str.len() != 2)

In [142]:
# every data point is good under the rule: no missing value no values are more than two characters
n_bad = int(mask_bad.sum())

In [143]:
# varible: op_carrier_fl_num    

s_raw_op = df["op_carrier_fl_num"].astype("string").str.strip()
s = s_raw_op.mask(s_raw_op.eq(""), pd.NA)

# numeric for validation
num = pd.to_numeric(s, errors="coerce")


mask_missing = s.isna()
mask_not_int = (~mask_missing) & (num.isna() | (num % 1 != 0))
int_val = num.round(0).astype("Int64")

# no more than 4 digits
mask_gt_4_digits = (~mask_missing) & (~mask_not_int) & ((int_val < 0) | (int_val > 9999))

mask_bad = mask_missing | mask_not_int | mask_gt_4_digits

# summary counts

summary = pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(mask_missing.sum())],
        "not_int": [int(mask_not_int.sum())],
        "gt_4_digits": [int(mask_gt_4_digits.sum())],
        "bad_total": [int(mask_bad.sum())],
    }
)
summary
# find one missing value


,total_rows,missing,not_int,gt_4_digits,bad_total
0,7079081,1,0,0,1


In [144]:
# identify the missing value loction
missing_obs = df.loc[mask_missing].copy()
missing_obs["_op_carrier_fl_num_clean"] = s[mask_missing].to_numpy()

missing_obs #by checking flight history online, the missing value is 2483

,year,month,day_of_month,day_of_week,fl_date,op_unique_carrier,op_carrier_fl_num,origin,origin_city_name,origin_state_nm,...,crs_elapsed_time,actual_elapsed_time,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,_op_carrier_fl_num_clean
4587670,2024,8,25,7,2024-08-25,F9,NaN,MIA,"Miami, FL",Florida,...,127.0,128.0,89.0,594.0,0,0,0,0,0,<NA>


In [145]:
# input value 2483 in dataset
fill_value = 2483


op = int_val.reindex(df.index)
op.loc[mask_bad.reindex(df.index, fill_value=False)] = fill_value

df["op_carrier_fl_num"] = op.astype("int64")


## Part 2-2

In [146]:
# varible: origin  


# non-missing and exactly 3 characters 
s_raw_or = df["origin"].astype("string").str.strip()
s = s_raw_or.mask(s_raw_or.eq(""), pd.NA)

mask_missing = s.isna()
mask_not_3 = (~mask_missing) & (s.str.len() != 3)
mask_bad = mask_missing | mask_not_3

# summary

pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(mask_missing.sum())],
        "not_3_chars": [int(mask_not_3.sum())],
        "bad_total": [int(mask_bad.sum())],
    }
)

# all good



,total_rows,missing,not_3_chars,bad_total
0,7079081,0,0,0


In [147]:
# origin_city_name: "New York, NY" "Minneapolis, MN"
# to "New York", "Minneapolis"
# origin_state: "NY", "MN"

# check missing value: no missing

s_raw = df["origin_city_name"].astype("string").str.strip()
s = s_raw.mask(s_raw.eq(""), pd.NA)

mask_missing = s.isna()


pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(mask_missing.sum())],
    }
)


,total_rows,missing
0,7079081,0


In [148]:
# split stste name to a new varible: origin_state_abbr


# origin_city_name: "New York, NY" "Minneapolis, MN"
# to "New York", "Minneapolis"
# origin_state: "NY", "MN"


s_raw1 = df["origin_city_name"].astype("string").str.strip()

origin_state_abbr = (
    s_raw1
    .str.extract(r",\s*([A-Za-z]{2})\s*$", expand=False)
    .str.upper()
)

origin_city_name = (
    s_raw1
    .str.replace(r",\s*[A-Za-z]{2}\s*$", "", regex=True)
    .str.strip()
)

df["origin_state_abbr"] = origin_state_abbr
df["origin_city_name"] = origin_city_name

# check
df[["origin_city_name", "origin_state_abbr"]].head()



,origin_city_name,origin_state_abbr
0,New York,NY
1,Minneapolis,MN
2,New York,NY
3,Richmond,VA
4,Detroit,MI


In [149]:
#check state name correction

us_state_abbr = [
    "AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA",
    "HI","ID","IL","IN","IA","KS","KY","LA","ME","MD",
    "MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ",
    "NM","NY","NC","ND","OH","OK","OR","PA","RI","SC",
    "SD","TN","TX","UT","VT","VA","WA","WV","WI","WY",
    "DC","PR", "VI", "TT"
]

abbr = df["origin_state_abbr"].astype("string").str.strip().str.upper()

mask_bad = abbr.isna() | (~abbr.isin(us_state_abbr))

pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(abbr.isna().sum())],
        "not_in_us_state_list": [int((~abbr.isna() & ~abbr.isin(us_state_abbr)).sum())],
        "bad_total": [int(mask_bad.sum())],
    }
)

# all good

,total_rows,missing,not_in_us_state_list,bad_total
0,7079081,0,0,0


In [150]:
# origin_state_nm


s_raw_sta = df["origin_state_nm"].astype("string").str.strip()
s = s_raw_sta.mask(s_raw_sta.eq(""), pd.NA)

mask_missing = s.isna()


pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(mask_missing.sum())],
    }
)

# no missig value

,total_rows,missing
0,7079081,0


In [151]:
full_names = [
    "Texas","California","Florida","Illinois","Georgia","New York","Colorado","North Carolina","Virginia",
    "Arizona","Nevada","Washington","Michigan","Pennsylvania","Tennessee","Massachusetts","New Jersey",
    "Minnesota","Missouri","Hawaii","Utah","Maryland","Ohio","Oregon","Kentucky","Louisiana","South Carolina",
    "Indiana","Wisconsin","Oklahoma","Puerto Rico","Alaska","Alabama","Idaho","Arkansas","New Mexico",
    "Nebraska","Montana","Iowa","Connecticut","North Dakota","Maine","Rhode Island","Kansas","South Dakota",
    "Mississippi","Wyoming","Vermont","U.S. Virgin Islands","New Hampshire","West Virginia",
    "U.S. Pacific Trust Territories and Possessions",
]



mask_bad = s_raw_sta.isna() | (~s_raw_sta.isin(full_names))

pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(s_raw_sta.isna().sum())],
        "not_in_us_state_list": [int((~s_raw_sta.isna() & ~s_raw_sta.isin(full_names)).sum())],
        "bad_total": [int(mask_bad.sum())],
    }
)

# all good

,total_rows,missing,not_in_us_state_list,bad_total
0,7079081,0,0,0


In [152]:
# varible: dest  


# non-missing and exactly 3 characters 
s_raw_d = df["dest"].astype("string").str.strip()
s = s_raw_d.mask(s_raw_d.eq(""), pd.NA)

mask_missing = s.isna()
mask_not_3 = (~mask_missing) & (s.str.len() != 3)
mask_bad = mask_missing | mask_not_3

# summary

pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(mask_missing.sum())],
        "not_3_chars": [int(mask_not_3.sum())],
        "bad_total": [int(mask_bad.sum())],
    }
)

# all good


,total_rows,missing,not_3_chars,bad_total
0,7079081,0,0,0


In [153]:
# dest_city_name: "Detroit, MI"
# to "Detroit"
# dest_state: "MI"


s_raw1 = df["dest_city_name"].astype("string").str.strip()

dest_state_abbr = (
    s_raw1
    .str.extract(r",\s*([A-Za-z]{2})\s*$", expand=False)
    .str.upper()
)

dest_city_name = (
    s_raw1
    .str.replace(r",\s*[A-Za-z]{2}\s*$", "", regex=True)
    .str.strip()
)

df["dest_state_abbr"] = dest_state_abbr
df["dest_city_name"] = dest_city_name

# check
df[["dest_city_name", "dest_state_abbr"]].head()



,dest_city_name,dest_state_abbr
0,Detroit,MI
1,Cleveland,OH
2,Richmond,VA
3,New York,NY
4,Milwaukee,WI


In [154]:
#check state name

us_state_abbr = [
    "AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA",
    "HI","ID","IL","IN","IA","KS","KY","LA","ME","MD",
    "MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ",
    "NM","NY","NC","ND","OH","OK","OR","PA","RI","SC",
    "SD","TN","TX","UT","VT","VA","WA","WV","WI","WY",
    "DC","PR", "VI", "TT"
]

abbr = df["dest_state_abbr"].astype("string").str.strip().str.upper()

mask_bad = abbr.isna() | (~abbr.isin(us_state_abbr))

pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(abbr.isna().sum())],
        "not_in_us_state_list": [int((~abbr.isna() & ~abbr.isin(us_state_abbr)).sum())],
        "bad_total": [int(mask_bad.sum())],
    }
)

#all good

,total_rows,missing,not_in_us_state_list,bad_total
0,7079081,0,0,0


In [155]:

# dest_state_nm


s_raw = df["dest_state_nm"].astype("string").str.strip()
s = s_raw.mask(s_raw.eq(""), pd.NA)

mask_missing = s.isna()


pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(mask_missing.sum())],
    }
)


full_names = [
    "Texas","California","Florida","Illinois","Georgia","New York","Colorado","North Carolina","Virginia",
    "Arizona","Nevada","Washington","Michigan","Pennsylvania","Tennessee","Massachusetts","New Jersey",
    "Minnesota","Missouri","Hawaii","Utah","Maryland","Ohio","Oregon","Kentucky","Louisiana","South Carolina",
    "Indiana","Wisconsin","Oklahoma","Puerto Rico","Alaska","Alabama","Idaho","Arkansas","New Mexico",
    "Nebraska","Montana","Iowa","Connecticut","North Dakota","Maine","Rhode Island","Kansas","South Dakota",
    "Mississippi","Wyoming","Vermont","U.S. Virgin Islands","New Hampshire","West Virginia",
    "U.S. Pacific Trust Territories and Possessions",
]



mask_bad = s_raw.isna() | (~s_raw.isin(full_names))

pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(s_raw.isna().sum())],
        "not_in_us_state_list": [int((~s_raw.isna() & ~s_raw.isin(full_names)).sum())],
        "bad_total": [int(mask_bad.sum())],
    }
)

# all good


,total_rows,missing,not_in_us_state_list,bad_total
0,7079081,0,0,0


## Part3

crs_dep_time

dep_time

dep_delay

taxi_out

wheels_off

wheels_on

taxi_in

crs_arr_time

arr_time

arr_delay

### part3_1

In [156]:
cols3_1 = ["crs_dep_time","dep_time","dep_delay","taxi_out","taxi_in","cancelled"]


In [157]:
print("Missing crs_dep_time:")
print(df["crs_dep_time"].isna().sum())

Missing crs_dep_time:
0


In [158]:
mask_cancel = df["cancelled"] == 1

df.loc[mask_cancel, ["dep_delay","taxi_out","taxi_in"]] = np.nan

In [159]:
s = pd.to_numeric(df["crs_dep_time"], errors="coerce")
s_str = s.dropna().astype(int).astype(str).str.zfill(4)

df["crs_dep_time_tmp"] = pd.NA

df.loc[s_str.index, "crs_dep_time_tmp"] = (
    pd.to_datetime(s_str, format="%H%M", errors="coerce")
      .dt.strftime("%H:%M")
)


In [160]:
s = pd.to_numeric(df["dep_time"], errors="coerce")
s_str = s.dropna().astype(int).astype(str).str.zfill(4)

df["dep_time_tmp"] = pd.NA

df.loc[s_str.index, "dep_time_tmp"] = (
    pd.to_datetime(s_str, format="%H%M", errors="coerce")
      .dt.strftime("%H:%M")
)


In [161]:
df["crs_dep_time"] = df["crs_dep_time_tmp"]
df.drop(columns=["crs_dep_time_tmp"], inplace=True)

df["dep_time"] = df["dep_time_tmp"]
df.drop(columns=["dep_time_tmp"], inplace=True)


In [162]:
df["dep_time"] = pd.to_datetime(
    df["dep_time"],
    format="%H:%M",
    errors="coerce"
).dt.time
df["crs_dep_time"] = pd.to_datetime(
    df["crs_dep_time"],
    format="%H:%M",
    errors="coerce"
).dt.time

In [163]:
crs = pd.to_numeric(df["crs_dep_time"], errors="coerce")
dep = pd.to_numeric(df["dep_time"], errors="coerce")

crs_h = crs // 100
crs_m = crs % 100
dep_h = dep // 100
dep_m = dep % 100

valid = (
    crs_h.between(0, 23) & crs_m.between(0, 59) &
    dep_h.between(0, 23) & dep_m.between(0, 59)
)

crs_min = crs_h * 60 + crs_m
dep_min = dep_h * 60 + dep_m

raw = dep_min - crs_min

mask = df["dep_delay"].notna() & valid

delta = df["dep_delay"] - raw

ok = (delta % 1440 == 0)

delay_mismatch = df.loc[mask & (~ok)]

print("dep_delay mismatch:", len(delay_mismatch))


dep_delay mismatch: 0


In [164]:

neg_taxi_out = df[df["taxi_out"] < 0]
neg_taxi_in  = df[df["taxi_in"] < 0]

print("\nNegative taxi_out:", len(neg_taxi_out))
print("Negative taxi_in:", len(neg_taxi_in))



Negative taxi_out: 0
Negative taxi_in: 0


### part3_2


In [165]:
s = pd.to_numeric(df["wheels_off"], errors="coerce")
s_str = s.dropna().astype(int).astype(str).str.zfill(4)

df["wheels_off_tmp"] = pd.NA

df.loc[s_str.index, "wheels_off_tmp"] = (
    pd.to_datetime(s_str, format="%H%M", errors="coerce")
      .dt.strftime("%H:%M")
)


In [166]:
s = pd.to_numeric(df["wheels_on"], errors="coerce")
s_str = s.dropna().astype(int).astype(str).str.zfill(4)

df["wheels_on_tmp"] = pd.NA

df.loc[s_str.index, "wheels_on_tmp"] = (
    pd.to_datetime(s_str, format="%H%M", errors="coerce")
      .dt.strftime("%H:%M")
)


In [167]:
s = pd.to_numeric(df["crs_arr_time"], errors="coerce")
s_str = s.dropna().astype(int).astype(str).str.zfill(4)

df["crs_arr_time_tmp"] = pd.NA

df.loc[s_str.index, "crs_arr_time_tmp"] = (
    pd.to_datetime(s_str, format="%H%M", errors="coerce")
      .dt.strftime("%H:%M")
)
     

In [168]:
s = pd.to_numeric(df["arr_time"], errors="coerce")
s_str = s.dropna().astype(int).astype(str).str.zfill(4)

df["arr_time_tmp"] = pd.NA

df.loc[s_str.index, "arr_time_tmp"] = (
    pd.to_datetime(s_str, format="%H%M", errors="coerce")
      .dt.strftime("%H:%M")
)
     

In [169]:
df["wheels_off"] = df["wheels_off_tmp"]
df.drop(columns=["wheels_off_tmp"], inplace=True)

df["wheels_on"] = df["wheels_on_tmp"]
df.drop(columns=["wheels_on_tmp"], inplace=True)

df["crs_arr_time"] = df["crs_arr_time_tmp"]
df.drop(columns=["crs_arr_time_tmp"], inplace=True)

df["arr_time"] = df["arr_time_tmp"]
df.drop(columns=["arr_time_tmp"], inplace=True)


In [170]:
df["wheels_off"] = pd.to_datetime(
    df["wheels_off"],
    format="%H:%M",
    errors="coerce"
).dt.time
df["wheels_on"] = pd.to_datetime(
    df["wheels_on"],
    format="%H:%M",
    errors="coerce"
).dt.time

df["arr_time"] = pd.to_datetime(
    df["arr_time"],
    format="%H:%M",
    errors="coerce"
).dt.time
df["crs_arr_time"] = pd.to_datetime(
    df["crs_arr_time"],
    format="%H:%M",
    errors="coerce"
).dt.time

In [171]:
crs = pd.to_numeric(df["crs_arr_time"], errors="coerce")
arr = pd.to_numeric(df["arr_time"], errors="coerce")

crs_h = crs // 100
crs_m = crs % 100
arr_h = arr // 100
arr_m = arr % 100

valid = (
    crs_h.between(0, 23) & crs_m.between(0, 59) &
    arr_h.between(0, 23) & arr_m.between(0, 59)
)

crs_min = crs_h * 60 + crs_m
arr_min = arr_h * 60 + arr_m

raw = arr_min - crs_min

mask = df["arr_delay"].notna() & valid

delta = df["arr_delay"] - raw

ok = (delta % 1440 == 0)

delay_mismatch = df.loc[mask & (~ok)]

print("arr_delay mismatch:", len(delay_mismatch))


arr_delay mismatch: 0


## Part 4

cancelled

cancellation_code

diverted 

In [172]:
# varible: cancelled

c_raw = df["cancelled"].astype("string").str.strip()
c = c_raw.mask(c_raw.eq(""), pd.NA)

c_num = pd.to_numeric(c, errors="coerce")

mask_c_missing = c.isna()
mask_c_not_int = (~mask_c_missing) & (c_num.isna() | (c_num % 1 != 0))
c_int = c_num.round(0).astype("Int64")
mask_c_not_01 = (~mask_c_missing) & (~mask_c_not_int) & (~c_int.isin([0, 1]))

mask_c_bad = mask_c_missing | mask_c_not_int | mask_c_not_01

cancelled_summary = pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(mask_c_missing.sum())],
        "not_int": [int(mask_c_not_int.sum())],
        "not_0_or_1": [int(mask_c_not_01.sum())],
        "bad_total": [int(mask_c_bad.sum())],
    }
)
cancelled_summary


# all good


,total_rows,missing,not_int,not_0_or_1,bad_total
0,7079081,0,0,0,0


In [173]:
mask_cancelled_1 = c_int.eq(1)

# related variables: set to NaN when cancelled == 1
rel_cols = [
    "dep_time", "dep_delay", "taxi_out", "wheels_off",
    "wheels_on", "taxi_in", "arr_time", "arr_delay","actual_elapsed_time", "air_time"
]

df.loc[mask_cancelled_1, rel_cols] = np.nan

In [174]:
# cancelled code


allowed = ["A", "B", "C", "D"]

# cancelled
c_raw = df["cancelled"].astype("string").str.strip()
c = c_raw.mask(c_raw.eq(""), pd.NA)
c_int = pd.to_numeric(c, errors="coerce").round(0).astype("Int64")


cc_raw = df["cancellation_code"].astype("string").str.strip()
cc = cc_raw.mask(cc_raw.eq(""), pd.NA).str.upper()

mask_cc_invalid_value = cc.notna() & (~cc.isin(allowed))

mask_cancelled0_code_present = c_int.eq(0) & cc.notna()
mask_cancelled1_code_missing = c_int.eq(1) & cc.isna()
mask_cancelled1_code_invalid = c_int.eq(1) & mask_cc_invalid_value

mask_bad = mask_cc_invalid_value | mask_cancelled0_code_present | mask_cancelled1_code_missing | mask_cancelled1_code_invalid

pd.DataFrame(
    {
        "total_rows": [len(df)],
        "invalid_code_value": [int(mask_cc_invalid_value.sum())],
        "cancelled_0_but_code_present": [int(mask_cancelled0_code_present.sum())],
        "cancelled_1_but_code_missing": [int(mask_cancelled1_code_missing.sum())],
        "cancelled_1_but_code_invalid": [int(mask_cancelled1_code_invalid.sum())],
        "bad_total": [int(mask_bad.sum())],
    }
)


# all good


,total_rows,invalid_code_value,cancelled_0_but_code_present,cancelled_1_but_code_missing,cancelled_1_but_code_invalid,bad_total
0,7079081,0,0,0,0,0


## Part 5

crs_elapsed_time  
actual_elapsed_time 

air_time            
distance

In [175]:

# crs_elapsed_time non-missing and >= 0

s_rawcrs = df["crs_elapsed_time"].astype("string").str.strip()
s = s_rawcrs.mask(s_rawcrs.eq(""), pd.NA)

num = pd.to_numeric(s, errors="coerce")

mask_missing = num.isna()
mask_negative = num.notna() & (num < 0)
mask_bad = mask_missing | mask_negative

pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(mask_missing.sum())],
        "negative": [int(mask_negative.sum())],
        "bad_total": [int(mask_bad.sum())],
    }
)

# one missing one negative


,total_rows,missing,negative,bad_total
0,7079081,1,1,2


In [176]:
cols_show = [c for c in ["crs_elapsed_time","crs_dep_time","crs_arr_time","origin","dest"] if c in df.columns]

#missing rows
missing_rows = df.loc[mask_missing, cols_show].copy()
missing_rows

,crs_elapsed_time,crs_dep_time,crs_arr_time,origin,dest
730836,NaN,18:00:00,22:52:00,BWI,SJU


In [177]:
cols_show = [c for c in ["crs_elapsed_time","crs_dep_time","crs_arr_time","origin","dest", "op_unique_carrier","op_carrier_fl_num"] if c in df.columns]

df.loc[mask_missing, "crs_elapsed_time"] = 292.0 # bwi to sju 18:00 to 22:52
df.loc[mask_missing, cols_show]

,crs_elapsed_time,crs_dep_time,crs_arr_time,origin,dest,op_unique_carrier,op_carrier_fl_num
730836,292.0,18:00:00,22:52:00,BWI,SJU,NK,2331


In [178]:
negative_rows = df.loc[mask_negative, cols_show].copy()
negative_rows 
# the flight time has problem, we don't have acess to find the flight info so remove

,crs_elapsed_time,crs_dep_time,crs_arr_time,origin,dest,op_unique_carrier,op_carrier_fl_num
1577980,-160.0,18:40:00,16:00:00,LAS,RNO,WN,336


In [179]:
df = df.drop(index=negative_rows.index)

In [180]:
# actual_elapsed_time

# >= 0
# if flight cancelled then = NaN already check above

s_rawcrs = df["actual_elapsed_time"].astype("string").str.strip()
s = s_rawcrs.mask(s_rawcrs.eq(""), pd.NA)

num = pd.to_numeric(s, errors="coerce")

mask_missing = num.isna()
mask_negative = num.notna() & (num < 0)

pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(mask_missing.sum())],
        "negative": [int(mask_negative.sum())]
    }
)

,total_rows,missing,negative
0,7079080,113813,0


In [181]:
c_raw = df["cancelled"].astype("string").str.strip()
c = c_raw.mask(c_raw.eq(""), pd.NA)
c_int = pd.to_numeric(c, errors="coerce").round(0).astype("Int64")

# cancelled = 0 but 'actual_elapsed_time' is missing
mask_bad = c_int.eq(0) & mask_missing


pd.DataFrame(
    {
        "total_rows": [len(df)],
        "cancelled_0_but_missing_actual_elapsed_time": [int(mask_bad.sum())],
    }
)

# remove where cancelled = 0 but 'actual_elapsed_time' is missing
df = df.drop(index=df.index[mask_bad])

In [182]:
#air_time

s_rawcrs = df["air_time"].astype("string").str.strip()
s = s_rawcrs.mask(s_rawcrs.eq(""), pd.NA)

num = pd.to_numeric(s, errors="coerce")

mask_missing = num.isna()
mask_negative = num.notna() & (num < 0)

pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(mask_missing.sum())],
        "negative": [int(mask_negative.sum())]
    }
)

,total_rows,missing,negative
0,7061581,96314,0


In [183]:
c_raw = df["cancelled"].astype("string").str.strip()
c = c_raw.mask(c_raw.eq(""), pd.NA)
c_int = pd.to_numeric(c, errors="coerce").round(0).astype("Int64")

# cancelled = 0 but 'actual_elapsed_time' is missing
mask_bad = c_int.eq(0) & mask_missing



pd.DataFrame(
    {
        "total_rows": [len(df)],
        "cancelled_0_but_missing_actual_elapsed_time": [int(mask_bad.sum())],
    }
)

# all good

,total_rows,cancelled_0_but_missing_actual_elapsed_time
0,7061581,0


In [184]:
# distance

s_rawdi = df["distance"].astype("string").str.strip()
s = s_rawdi.mask(s_rawdi.eq(""), pd.NA)

num = pd.to_numeric(s, errors="coerce")

mask_missing = num.isna()
mask_negative = num.notna() & (num < 0)

pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(mask_missing.sum())],
        "negative": [int(mask_negative.sum())]
    }
)

# all good

,total_rows,missing,negative
0,7061581,0,0


## Part 6

carrier_delay   
weather_delay     
nas_delay         
security_delay    
late_aircraft_delay 

In [185]:
cols6 = ["carrier_delay", "weather_delay", "nas_delay", "security_delay", "late_aircraft_delay"]


In [186]:
neg_mask = (df[cols6] < 0).any(axis=1)

neg_rows = df.loc[neg_mask, cols6]

print("Rows with negative delay components:", len(neg_rows))

Rows with negative delay components: 0


In [187]:
mask_arr = df["arr_delay"] > 0

sum_comp = df[cols6].sum(axis=1)

arr_delay_mismatch = mask_arr & (sum_comp != df["arr_delay"])

print("arr_delay sum mismatch rows:", arr_delay_mismatch.sum())


arr_delay sum mismatch rows: 1072038


In [188]:
df["arr_delay_consistent"] = True
df.loc[mask_arr, "arr_delay_consistent"] = (
    sum_comp[mask_arr] == df.loc[mask_arr, "arr_delay"]
)

In [189]:

print("arr_delay > 0 rows:", mask_arr.sum())
print("consistent rows:",
      (mask_arr & (sum_comp == df["arr_delay"])).sum())
print("inconsistent rows:",
      (mask_arr & (sum_comp != df["arr_delay"])).sum())


arr_delay > 0 rows: 2522008
consistent rows: 1449970
inconsistent rows: 1072038


In [190]:
has_na_component = df[cols6].isna().any(axis=1)

(mask_arr & (sum_comp != df["arr_delay"]) & has_na_component).sum()


np.int64(0)

In [191]:
less = mask_arr & (sum_comp < df["arr_delay"])
greater = mask_arr & (sum_comp > df["arr_delay"])

print("sum < arr_delay:", less.sum())
print("sum > arr_delay:", greater.sum())


sum < arr_delay: 1072038
sum > arr_delay: 0


In [192]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7061581 entries, 0 to 7079080
Data columns (total 38 columns):
 #   Column                Dtype         
---  ------                -----         
 0   year                  int64         
 1   month                 int64         
 2   day_of_month          int64         
 3   day_of_week           int64         
 4   fl_date               datetime64[ns]
 5   op_unique_carrier     object        
 6   op_carrier_fl_num     int64         
 7   origin                object        
 8   origin_city_name      string        
 9   origin_state_nm       object        
 10  dest                  object        
 11  dest_city_name        string        
 12  dest_state_nm         object        
 13  crs_dep_time          object        
 14  dep_time              object        
 15  dep_delay             float64       
 16  taxi_out              float64       
 17  wheels_off            object        
 18  wheels_on             object        
 19  taxi_

In [193]:
from pathlib import Path

out_dir = Path("data")
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / "flights_clean.csv"
df.to_csv(out_path, index=False)

size_mib = out_path.stat().st_size / (1024 * 1024)
print(f"saved: {out_path} | rows={len(df):,} | cols={df.shape[1]} | size={size_mib:.2f} MiB")


saved: data/flights_clean.csv | rows=7,061,581 | cols=38 | size=1343.68 MiB
